# Abstract
计算Pt（111）表面氧原子的扩散能垒和应变调控作用


In [58]:
from atoms_model import *

In [59]:
# packages

import os
import math
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from os import system
from os.path import join
from copy import deepcopy
from genericpath import exists

from ase import Atoms
from ase.io import read
from ase.visualize import view
from ase.build import add_adsorbate, molecule
# from ase.io.lammpsdata import write_lammps_data

In [60]:
# 设置全局字体为 Times New Roman
plt.rcParams["font.family"] = "serif"          # 使用衬线字体
plt.rcParams["font.serif"] = ["Times New Roman"]  # 指定 Times New Roman

plt.rcParams["mathtext.fontset"] = "custom"
plt.rcParams["mathtext.rm"] = "Times New Roman"
plt.rcParams["mathtext.it"] = "Times New Roman:italic"
plt.rcParams["mathtext.bf"] = "Times New Roman:bold"

In [61]:
savepath = '.\\PtO' # 保存路径根目录
input_dir = join(savepath, 'input') # 输入目录
output_dir = join(savepath, 'output') # 输出目录
md_dir = join(savepath, 'md') # 输出目录

input_share = '.\\PtORR\\input'
create_folder(savepath, is_delete=False)
create_folder(input_dir, is_delete=False)
create_folder(output_dir, is_delete=False)
create_folder(md_dir, is_delete=False)

folder .\PtO exists
folder .\PtO\input exists
folder .\PtO\output exists
folder .\PtO\md exists


# Model

In [62]:
Pt_bulk_path = join(input_share,'Pt_bulk_origin')
if os.path.exists(join(input_dir,'Pt_bulk_origin')):
    Pt_bulk = read_one_car(join(input_dir,'Pt_bulk_origin')) # 读取Pt_bulk数据并重复三次
    lattice = Pt_bulk.get_cell()[-1,-1]
else:
    lattice = 3.97 # 默认Pt晶格常数
lattice # 获得DFT计算的Pt晶格常数

3.967271975756752

In [63]:
# 创建Pt111slab模型
recreate = False  # 是否重复Pt_bulk
filename = join(input_dir, 'Pt.cfg')
if not exists(filename):
    print( f'File dose not exist: {filename}' )
    recreate = True
if recreate:
    try:
        del_file(filename)
        handle_line = f'atomsk --create fcc {lattice} Pt orient [-1-12] [1-10] [111] {filename}'
        print(handle_line)
        system(handle_line)
    except:
        print('atomsk is uninstall: websit\n atomsk.univ-lille.fr')

Pt_bulk = read(filename)
print(Pt_bulk.get_cell())
# out_poscar(Pt_bulk, join(output_dir, 'Pt_bulk'))  # 输出POSCAR文件

Cell([4.85889601, 2.80528492, 6.87151663])


In [64]:
Pt_bulk = read_one_car(join(input_share, 'Pt_bulk_copy'))  # 读取Pt_bulk数据
Pt_bulk_e = read_one_car(join(input_share, 'Pt_bulk_copy'))  # 读取Pt_bulk数据

print(Pt_bulk.get_cell())

Cell([4.85889601, 2.80528492, 6.87151663])


In [65]:
Pt_slab = read_one_car(join(input_share, 'Pt_slab_copy'))  # 读取Pt_slab数据
Pt_slab_e = read_one_car(join(input_share, 'Pt_slab_copy'), car='OSZICAR')  # 读取Pt_slab数据

Pt_slab = fix_layers_atoms(Pt_slab, {(-1,-1,-1):[-1,-2]})
Pt_slab = Pt_slab.repeat((2,4,1))
Pt_slab_e = Pt_slab_e*2*4

E0 value: -1 F   -.46274982E+02


# O<sub>2</sub> diffusion

## Oadsorption